In [2]:
import cv2
import numpy as np

In [8]:
cap = cv2.VideoCapture('./Project_outdoor video1.mov')
bgs = cv2.createBackgroundSubtractorKNN()

fps = cap.get(cv2.CAP_PROP_FPS)
delay = round(300/fps)

if not cap.isOpened():
    print("Camera open failed!")
    exit()
    

while True:
    # 카메라로부터 프레임을 정상적으로 받아오면 ret에는 True,
    # frame에는 해당 프레임이 저장됨
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # 현재 프레임 반전
    resize_frame = cv2.resize(frame, (0,0), fx=3, fy=3, interpolation = cv2.INTER_LANCZOS4)
    
    fgmask = bgs.apply(resize_frame)
    
    mask = np.zeros((fgmask.shape[0]+2, fgmask.shape[1]+2), np.uint8)
    _, fgmask_fill = cv2.threshold(fgmask, 170, 255, cv2.THRESH_BINARY)
    
    
    fgmask_blur = cv2.medianBlur(cv2.medianBlur(fgmask_fill, 5), 5)
    final_mask = fgmask_blur.copy()
    cv2.floodFill(final_mask, mask, (0,0), 255)
    final = cv2.bitwise_xor(~final_mask, fgmask_blur)
    final = cv2.dilate(final, np.ones((3,3), np.uint8), iterations=3)
    
    #채널별로 엣지까지 제거한 결과
    final_B = cv2.medianBlur(final, 3)&resize_frame[:,:,0]
    final_G = cv2.medianBlur(final, 3)&resize_frame[:,:,1]
    final_R = cv2.medianBlur(final, 3)&resize_frame[:,:,2]
    
    #medianBlur 추가해서 salt noise 제거
    #final_end = cv2.medianBlur(cv2.merge((final_B,final_G,final_R)), 3)
    final_end = cv2.merge((final_B,final_G,final_R))
    
    cv2.imshow('car Video', final_end)
    
    # 일정 시간 기다린 후 다음 프레임 처리.
    # 만약 ESC 키를 누르면 while 루프 종료
    if cv2.waitKey(delay) == 27:
        break

# 사용한 자원 해제
cap.release()
cv2.destroyAllWindows()
